# Microsoft Movie Project 

## Overview

## Business Problem

Microsoft Studios wants to start the production of films. In order to be successful doing so they reached out to me and asked for recommendations on how they can be successful in doing so. 

## Data Understanding 

# Importing Data 

In [ ]:
import pandas as pd 
import sqlite3
from matplotlib import pyplot as plt
import numpy as np 
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns

In [ ]:
!ls zippedData/

## Movie Gross 

In [ ]:
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')

In [ ]:
bom.head(5)

## Movies

In [ ]:
movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')

In [ ]:
movies.head(5)

## Movie Budgets 

In [ ]:
movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [ ]:
movie_budgets.head(5)

# Questions

- What studio had the greatest total gross earnings? 
- What type of relationship is there between production budget and total gross earnings?
- What month of release date had the greatest gross earnings? 

## Q1: What studio had the greatest total gross earnings

In [ ]:
len(bom)

In [ ]:
bom.info()

In [ ]:
bom.head()

In [ ]:
'''
This line of code was used to fill in null values under the domestic_gross columns with 0. 
'''
domestic_gross = bom.loc[:,"domestic_gross"].fillna(0)
print(domestic_gross)

Filling null values with 0 is resonable to assume that they did not have any gross earnings. 

In [ ]:
print(foreign_gross)

In [ ]:
bom['total_gross'] = foreign_gross + domestic_gross 

In [ ]:
bom

In [ ]:
'''
Line of code to create a new column where total_gross is now displayed in millions rounded to the second
decimal place which is easier to read than total_gross in scientific method

'''
bom['total_gross_in_millions'] = bom['total_gross'].div(1000000).round(2)

In [ ]:
bom

In [ ]:
studios = bom.groupby('studio')['total_gross_in_millions'].agg(['mean','count']).reset_index()

In [ ]:
studios

In [ ]:
studios_clean = studios.loc[studios['count']>3].sort_values('mean', ascending = False)

In [ ]:
studios_clean_ten = studios_clean.head(10)

In [ ]:
'''
Lines of code to rename the studios under the Studio's column
source:https://www.geeksforgeeks.org/add-a-new-column-in-pandas-data-frame-using-a-dictionary/
'''

#data_frame = pd.DataFrame([[i] for i in range(7)], columns =['data'])
  
# Introducing weeks as dictionary
studios_dict = {'P/DW':'Pixar / Dreamworks', 'BV':'Buena Vista', 'WB (NL)':'New Line Cinema', 'Fox':'Fox', 
'WB':'Warner Bros.','Sony':'Sony', 'Uni.':'Universal', 'Par.':'Paramount','Sum.':'Summit Entertainment', 'LG/S':'Lifes Good / Sony'}
  
# Mapping the dictionary keys to the data frame.
studios_clean_ten['studio'] = studios_clean_ten['studio'].map(studios_dict)


In [ ]:
'''
These lines of code plot the top 20 studios with the greatest
gross earnings and that produced at least 3 films. 
'''

fig, ax = plt.subplots()
ax.barh(y= studios_clean_ten['studio'], width = studios_clean_ten['mean'] )
ax.set_title("Studio's Gross Earnings")
ax.set_xlabel("Gross Earnings (in millions)")
ax.set_ylabel("Studio")
ax.invert_yaxis()
#plt.show()
plt.savefig('Images/studios_gross_earnings.png', bbox_inches = 'tight')

## Q2: What type of relationship is there between production budget and total gross earnings?

In [ ]:
movie_budgets

In [ ]:
movie_budgets.loc[:,"domestic_gross"]

In [ ]:
domestic_gross_clean = movie_budgets.loc[:,"domestic_gross"].str.replace(",","").str.replace("$","").astype(float).fillna(0)

In [ ]:
movie_budgets.loc[:,"worldwide_gross"]

In [ ]:
worldwide_gross_clean = movie_budgets.loc[:,"worldwide_gross"].str.replace(",","").str.replace("$","").astype(float).fillna(0)

In [ ]:
movie_budgets['total_gross'] = domestic_gross_clean + worldwide_gross_clean

In [ ]:
movie_budgets.head()

In [ ]:
movie_budgets['total_gross_in_millions'] = movie_budgets['total_gross'].div(1000000).round(2)

In [ ]:
movie_budgets.head()

In [ ]:
production_budget_clean = movie_budgets["production_budget"].str.replace(",","").str.replace("$","").astype(float)

In [ ]:
movie_budgets['production_budget_clean'] = production_budget_clean

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(y= movie_budgets['total_gross']/1000000, 
           x = movie_budgets['production_budget_clean']/1000000)
ax.set_title("Production Budget vs. Total Gross Earnings ")
ax.set_xlabel("Production Budget (in million)")
ax.set_ylabel("Gross Earnings (in millions)")
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
ax.xaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
#ax.set_xlim([0,3500000000])
#plt.show()
ax.vlines(175, 175,3500)
ax.hlines(175, 175,450)

plt.savefig('Images/production_budget_vs_total_gross_earnings.png', bbox_inches = 'tight')

In [ ]:
high_budget_gross = movie_budgets.loc[(movie_budgets['production_budget_clean']>175000000)
                  &(movie_budgets['total_gross']>175000000)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.regplot(y= high_budget_gross['total_gross']/1000000, 
           x = high_budget_gross['production_budget_clean']/1000000, ax=ax)
ax.set_title("Production Budget vs. Total Gross Earnings ")
ax.set_xlabel("Production Budget (in million)")
ax.set_ylabel("Gross Earnings (in millions)")
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
ax.xaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
ax.set_xlim([0,450])
ax.set_ylim([0,3750])
ax.vlines(175, 175,3500)

plt.savefig('Images/production_budget_vs_total_gross_earnings_zoom.png', bbox_inches = 'tight')

## Q3: What month of release date had the greatest gross earnings?

In [ ]:
movie_budgets.head()

In [ ]:
movie_budgets['release_date'] = pd.to_datetime(movie_budgets['release_date'])

In [ ]:
movie_budgets.head()

In [ ]:
movie_budgets['year'] = movie_budgets['release_date'].dt.year

In [ ]:
movies.duplicated(['title','year']).sum()

In [ ]:
movies_clean = movies.sort_values(by="popularity", ascending=False).drop_duplicates(['title','year'])

In [ ]:
len(movies_clean)

In [ ]:
movie_budgets.duplicated(['movie','year']).sum()

In [ ]:
def movie_budgets_clean(x):
    '''
    A function to clean up money columns
    This removes '$' and ',', and turns the string into a float
    It is designed to be used with .apply on each column
    
    Input: x (string cell in a dataframe)
    Output: float version of x
    '''
    return float(x.replace("$", "").replace(",",""))

In [ ]:
for column in['production_budget','domestic_gross','worldwide_gross']:
    movie_budgets[column] = movie_budgets[column].apply(movie_budgets_clean)

In [ ]:
movie_budgets['month'] = movie_budgets['release_date'].dt.month

In [ ]:
movie_budgets

In [ ]:
budget_months = movie_budgets.groupby('month')['total_gross_in_millions'].agg(['mean','count']).reset_index()

In [ ]:
budget_months

In [ ]:
fig, ax = plt.subplots()
ax.bar(budget_months['month'],budget_months['mean'])
ax.set_title("Average Total Gross per Month")
ax.set_xlabel("Month")
ax.set_ylabel("Average Total Gross Earnings")

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(budget_months['month'],budget_months['count'])
ax.set_title("Number of Movies per Month ")
ax.set_xlabel("Month")
ax.set_ylabel("Number of Movies")

plt.show()

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.bar(budget_months['month'],budget_months['count'], color=color, alpha=.5)
ax1.set_title("Number of Movies per Month ")
ax1.set_xlabel("Month")
ax1.set_ylabel("Number of Movies", color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.plot(budget_months['month'],budget_months['mean'], color=color)
ax2.set_ylabel("Average Total Gross Earnings", color=color)
 # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim([0,250])
fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()
plt.savefig('Images/number_of_movies_per_month.png', bbox_inches = 'tight')

In [ ]:
len(movies)

In [ ]:
len(movie_budgets)

In [ ]:
len(bom)

## Exploratory Data Analysis 

1. What studio had the greatest total gross earnings?

2. What type of relationship is there between production budget and total gross earnings?

3. What month of release date had the greatest gross earnings?

## Limitations 

- 
- 
- 

## Conclusions / Recommendations

This analysis leads to three recommendations for increasing Microsoft's gross earnings :

1. 
2.
3.

## Next Steps 

Further analyses could yield additional insights to further increase Microsoft's gross earnings:

1. 
2.
3.